In [ ]:
import numpy as np
import pandas as pd
import dionysus
import matplotlib.pyplot as plt
import os

In [ ]:
# global parameters
threshold = 0.8
p = 2

In [ ]:
def compute_graph(data):
    corr_matrix = data.corr()
    graph = corr_matrix > threshold
    return graph

def compute_simplicial_complex(graph):
    n = len(graph)
    simplicial_complex = []
    prev = []
    nxt = []
    simplices = set()
    for i in range(n):
        simplicial_complex.append(([i], 0))
        prev.append([i])
    for k in range(1, n + 1):
        for i in range(n):
            for s in prev:
                if i in s:
                    continue
                flag = 1
                for elem in s:
                    if not graph[i][elem]:
                        flag = 0
                        break
                if flag:
                    new_simplex = tuple(s) + tuple([i])
                    new_simplex = tuple(sorted(list(new_simplex)))
                    if not new_simplex in simplices:
                        simplices.add(new_simplex)
                        nxt.append(new_simplex)
                        simplicial_complex.append((list(new_simplex), k))
        prev = nxt
        nxt = []
        simplices = set()
    return simplicial_complex

def compute_persistence_diag(simplicial_complex):
    f = dionysus.Filtration()
    for vertices, time in simplicial_complex:
        f.append(dionysus.Simplex(vertices, time))
    m = dionysus.homology_persistence(f)
    dgms = dionysus.init_diagrams(m, f)
    return dgms

def compute(data):
    graph = compute_graph(data)
    simplicial_complex = compute_simplicial_complex(graph)
    diag = compute_persistence_diag(simplicial_complex)
    return diag

In [ ]:
diags = []
directory = 'Training'
for filename in os.listdir(directory):
    if filename[:3] == 'sub':
        print('./' + filename + '/timeseries_aal.csv')
        data = pd.read_csv('./Training/' + filename + '/timeseries_aal.csv', header = None).T
        diags.append(compute(data))

In [ ]:
cnt = 0
for dgms in diags:
    f = open('./homology_data/' + str(cnt), 'w')
    cnt += 1
    for i, dgm in enumerate(dgms):
        for pt in dgm:
            print(i, pt.birth, pt.death, file = f)

In [ ]:
m = len(diags)
distances = np.zeros((m, m))
for i in range(m):
    print(i)
    for j in range(m):
        try:
            distances[i][j] = dionysus.wasserstein_distance(diags[i][2], diags[j][2], p)
        except Exception as e:
            distances[i][j] = 1e9
f = open('distances2', 'w')
for row in distances:
    for elem in row:
        print(elem, end = ' ', file = f)
    print(file = f)

In [ ]:
m = len(diags)
distances = np.zeros((m, m))
for i in range(m):
    print(i)
    for j in range(m):
        try:
            distances[i][j] = dionysus.wasserstein_distance(diags[i][3], diags[j][3], p)
        except Exception as e:
            distances[i][j] = 1e9
f = open('distances3', 'w')
for row in distances:
    for elem in row:
        print(elem, end = ' ', file = f)
    print(file = f)

In [ ]:
f = open('distances2', 'r')
X = []
for line in f.readlines():
    X.append(list(map(float, line.split())))

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
for i in range(len(X)):
    for j in range(len(X[i])):
        if X[i][j] == float('inf'):
            X[i][j] = 10000

In [ ]:
model = DBSCAN(metric = 'precomputed', eps = 3, min_samples = 2)
model.fit(X)

In [ ]:
preds = model.labels_

In [ ]:
y_true = []
for filename in os.listdir('Training'):
    if filename[:3] == 'sub':
        data = pd.read_csv('./Training/' + filename + '/phenotypic.csv')
        if data['DX'][0] == 'Control':
            y_true.append(0)
        else:
            y_true.append(1)
        

In [ ]:
cluster = -1
np.sum(np.array(y_true)[preds == cluster]) / len(np.array(y_true)[preds == cluster])